In [1]:
#Парсинг Владивосток

In [10]:
#Импортируем необходиимых библиотек для реализации парсинга
from selenium import webdriver
from selenium_stealth import stealth # делает похожим на настоящего пользователя
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np

from selenium.webdriver.chrome.service import Service

In [12]:
import selenium
selenium.__version__

'4.15.2'

In [14]:
!pip install selenium == 4.2.0

Defaulting to user installation because normal site-packages is not writeable


ERROR: Invalid requirement: '=='


In [ ]:
#Выгружаем данные (характеристики, адрес, текст объявления)
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("start_maximized")

driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")

#driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
#driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent.random})

stealth(driver,
#        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
        languages=["en-US", "en"],
        vendor = "Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=False, #True 
        run_on_insecure_origins=False
        )

link_all = []

p = 5 # количество страниц

for page in tqdm(range(1, p, 1), 'Обработанно страниц'):
    
    #url = f"https://www.avito.ru/vladivostok/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?cd=1&p={page}" #вторичка
    url = f"https://www.avito.ru/vladivostok/kvartiry/prodam/novostroyka-ASgBAgICAkSSA8YQ5geOUg?cd=1&f=ASgBAgECA0SSA8YQ5geOUsDBDbr9NwFFuMENFHsiZnJvbSI6bnVsbCwidG8iOjF9&p={page}" #дом сдан
    
    driver.get(url)
    sleep(2) #спим 2 секунды (прогрузка страницы)
    
    #выгружаем ссылку по всем квартирам
    blocks = driver.find_elements_by_class_name('iva-item-titleStep-pdebR')
    for block in blocks:
        link = block.find_element_by_tag_name('a').get_attribute("href")
        link_all.append(link)

In [18]:
len(link_all)

141

In [19]:
link_all

['https://www.avito.ru/vladivostok/kvartiry/kvartira-studiya_356_m_1417_et._2411230367',
 'https://www.avito.ru/vladivostok/kvartiry/kvartira-studiya_143_m_79_et._3370963176',
 'https://www.avito.ru/vladivostok/kvartiry/3-k._kvartira_658_m_210_et._3366181643',
 'https://www.avito.ru/vladivostok/kvartiry/2-k._kvartira_47_m_29_et._1871528830',
 'https://www.avito.ru/vladivostok/kvartiry/dolya_v_3-k._kvartire_72_m_99_et._3577474223',
 'https://www.avito.ru/vladivostok/kvartiry/kvartira-studiya_27_m_28_et._3629780349',
 'https://www.avito.ru/vladivostok/kvartiry/1-k._kvartira_326_m_99_et._3328522645',
 'https://www.avito.ru/vladivostok/kvartiry/2-k._kvartira_637_m_424_et._3604252165',
 'https://www.avito.ru/vladivostok/kvartiry/1-k._kvartira_696_m_226_et._3008607913',
 'https://www.avito.ru/vladivostok/kvartiry/1-k._kvartira_906_m_926_et._3008735936',
 'https://www.avito.ru/vladivostok/kvartiry/2-k._kvartira_613_m_36_et._3168527027',
 'https://www.avito.ru/vladivostok/kvartiry/3-k._kvartir

In [32]:
#Выгружаем данные (характеристики, адрес, текст объявления)
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")


stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

t1 = [] #список
a = 0

for lnk in tqdm(link_all, 'Сбор данных'):
    try:
        a = a + 1 #счетчик
        t1.clear() #очищаем список значений
        
        driver.get(lnk) #открываем ссылку
        sleep(4) #спим 4 секунды (прогрузка страницы)
        
        #ТАБЛИЦА 1
        table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
        k_1 = len(table_1) #количество строк в таблице 1

        for j in range(0, k_1):
            table_t1 = table_1[j].text.split(':') #выгружаем текст и разделяем его на 2 части по ":"
            t1.append(table_t1) #складываем в список
        
        #ТАБЛИЦА 2
        table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H').find_elements_by_tag_name('li') #таблица 2
        k_2 = len(table_2) #количество строк в таблице 2

        for j in range(0, k_2):
            table_t2 = table_2[j].text.split(':') #выгружаем текст и разделяем его на 2 части по ":"
            t1.append(table_t2) #складываем в список
        
        #АДРЕС
        address = driver.find_element_by_class_name('style-item-address-KooqC').find_element_by_tag_name('span').text #адрес
        addr = ['Адрес', address] #создаем список для адреса
        t1.append(addr) #складываем в список
        
        #ТЕКСТ ОБЪЯВЛЕНИЯ
        text_o = driver.find_element_by_class_name('style-item-description-pL_gy').text #текст объявления
        to = ['Текст', text_o] #создаем список для текста объявления
        t1.append(to) #складываем в список
         
        #ЦЕНА
        price = driver.find_element_by_class_name('style-price-value-main-TIg6u').find_element_by_tag_name('span').get_attribute("content")
        price_1 = ['Цена', price] #создаем список для цены
        t1.append(price_1) #складываем в список
        
        #ССЫЛКА
        lnk_1 = ['Ссылка', lnk] #создаем список для ссылки
        t1.append(lnk_1) #складываем в список
        
        #ДАТА И ПРОСМОТРЫ
        data = driver.find_element_by_class_name('style-item-footer-text-LEjEe').find_elements_by_tag_name('span')
        dt = ['Дата', data[1].text] #создаем список
        v = ['Просмотры', data[3].text]  #создаем список
        t1.append(dt) #складываем в список
        t1.append(v) #складываем в список
        
        #ПРОДАВЕЦ
        seller_av = driver.find_element_by_class_name('style-seller-info-col-PETb_').find_elements_by_tag_name('div')
        seller = ['Продавец', seller_av[2].text] #создаем список для цены
        t1.append(seller)                        #складываем в список
        
        df = pd.DataFrame(t1)
        df = df.set_index(0)    

        if a == 1:
            df_result = df
        if a > 1:
            df_result = pd.concat([df_result, df], axis=1)
    
    except:
        winsound.Beep(2000, 2000) #громкий звук в 2 сек
        print('Сплю минуту')
        pages_lnk.append(lnk)     #страницы, которые не прошли
        sleep(60)                 #спим минуту для разблокировки руками
        #continue

df_result_copy = df_result.copy()
df_result_copy = df_result_copy.transpose()

#Сохраняем в Excel
writer = pd.ExcelWriter('Квартиры_Владивосток_вторич_avito.xlsx')
df_result_copy.to_excel(writer, 'marks')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
Сбор данных:   0%|                                                                             | 0/141 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppD

Сбор данных:   2%|█▍                                                                   | 3/141 [00:44<27:44, 12.06s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:38: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:   4%|██▉                                                                  | 6/141 [01:06<18:44,  8.33s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:38: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:46: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  address = driver.find_element_by_class_name('style-item-address-KooqC').find_element_by_tag_name('span').text #адрес
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:46: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  address = driver.find_element_by_class_name('style-item-address-KooqC').find_element_by_tag_name('span').text #адрес
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:51: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  text_o = driver.find_element_by_class_name('style-item-description-pL_gy').text #текст объявления
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:56: DeprecationWarning: fi

Сбор данных:   8%|█████▎                                                              | 11/141 [01:34<12:51,  5.94s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:38: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

...

Сбор данных:  91%|█████████████████████████████████████████████████████████████▎     | 129/141 [12:43<01:10,  5.88s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:38: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

Сбор данных:  94%|██████████████████████████████████████████████████████████████▋    | 132/141 [13:00<00:52,  5.80s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:38: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:65: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  data = driver.find_element_by_class_name('style-item-footer-text-LEjEe').find_elements_by_tag_name('span')
Сбор данных:  96%|████████████████████████████████████████████████████████████████▏  | 135/141 [13:18<00:34,  5.74s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elemen

C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:46: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  address = driver.find_element_by_class_name('style-item-address-KooqC').find_element_by_tag_name('span').text #адрес
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:46: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  address = driver.find_element_by_class_name('style-item-address-KooqC').find_element_by_tag_name('span').text #адрес
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:51: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  text_o = driver.find_element_by_class_name('style-item-description-pL_gy').text #текст объявления
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:56: DeprecationWarning: fi

Сбор данных:  99%|██████████████████████████████████████████████████████████████████▌| 140/141 [13:46<00:05,  5.61s/it]C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:30: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  table_1 = driver.find_element_by_class_name('params-paramsList-zLpAu').find_elements_by_tag_name('li') #таблица 1
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3381898819.py:38: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  table_2 = driver.find_element_by_class_name('style-item-params-list-vb1_H

In [14]:
#Выгружаем данные (характеристики, адрес, текст объявления)
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")


stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )


url = f"https://www.avito.ru/vladivostok/kvartiry/1-k._kvartira_36_m_715_et._3349586281"
driver.get(url)

sleep(4) #спим 4 секунды (прогрузка страницы)

#ДАТА И ПРОСМОТРЫ
data = driver.find_element_by_class_name('style-item-footer-text-LEjEe').find_elements_by_tag_name('span')
dt = data[1].text
v = data[3].text
#dt = ['Дата', data] #создаем список
#t1.append(dt) #складываем в список



C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3447801061.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3447801061.py:25: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  data = driver.find_element_by_class_name('style-item-footer-text-LEjEe').find_elements_by_tag_name('span')
C:\Users\skapr\AppData\Local\Temp\ipykernel_14688\3447801061.py:25: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  data = driver.find_element_by_class_name('style-item-footer-text-LEjEe').find_elements_by_tag_name('span')


In [15]:
dt

'· 20 ноября в 19:39'

In [16]:
v

'370 просмотров'